In [3]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
from typing import Optional, Tuple
import warnings
warnings.filterwarnings("ignore")

# =============================================================================
# Core EVDS Data Fetching Function
# =============================================================================

def evds_serie_market_all(
    series_code: str,
    start: str,
    end: str,
    datagroup: str = "",
    category: str = "",
    idx: int = 0,
    page_size: int = 20,
    aggregation_type: str = "last",
    frequency: str = "YEARWEEK",
    date_format_value: Optional[str] = None,
    verbose: bool = False,
    session: Optional[requests.Session] = None,
) -> pd.DataFrame:
    
    s = session or requests.Session()
    s.headers.update({
        "Accept": "*/*",
        "Accept-Language": "tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "User-Agent": "Mozilla/5.0",
        "X-Requested-With": "XMLHttpRequest",
        "Referer": "https://evds2.tcmb.gov.tr/index.php?/evds/serieMarket",
        "Origin": "https://evds2.tcmb.gov.tr",
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    })

    # ---- İlk GET (session açılışı) try/except ----
    try:
        g = s.get("https://evds2.tcmb.gov.tr/index.php?/evds/serieMarket", timeout=30)
        if verbose:
            print("GET status:", g.status_code)
    except requests.RequestException as e:
        if verbose:
            print(f"GET isteğinde hata oluştu: {e}")
        return pd.DataFrame()

    select_code = f"{series_code}-{idx}"
    
    if date_format_value is None:
        date_format_value = "yyyy-ww"

    all_rows = []
    total_count = None
    skip = 0
    page_no = 1

    while True:
        payload = {
            "orderby": "Tarih desc",
            "thousand": 1,
            "decimal": 2,
            "sort": "Tarih#true",
            "frequency": frequency,
            "aggregationType": aggregation_type,
            "formula": 0,
            "graphicType": "0",
            "skip": skip,
            "take": page_size,
            "select": select_code,
            "startDate": start,
            "endDate": end,
            "categories": category,
            "mongoAdresses": "evds",
            "datagroupString": datagroup,
            "obsCountEnabled": "",
            "obsCount": "",
            "userId": "",
            "dateFormatValue": date_format_value,
            "customFormula": "null",
            "excludedSeries": "null",
        }

        # ---- POST try/except ----
        try:
            r = s.post("https://evds2.tcmb.gov.tr/EVDSServlet", data=payload, timeout=60)
            if verbose:
                print(f"PAGE {page_no} | skip={skip} | status={r.status_code}")
            r.raise_for_status()
        except requests.RequestException as e:
            if verbose:
                print(f"POST isteğinde hata oluştu (PAGE {page_no}, skip={skip}): {e}")
            break

        try:
            data = r.json()
        except ValueError:
            break

        if isinstance(data, list) and data:
            if total_count is None:
                total_count = data[0].get("totalCount", len(data))
            items_list = [row.get("items", {}) for row in data]
        elif isinstance(data, dict) and "items" in data:
            if total_count is None:
                total_count = data.get("totalCount", len(data["items"]))
            items_list = data["items"]
        else:
            items_list = []

        if not items_list:
            break

        all_rows.extend(items_list)
        got = len(items_list)
        skip += got
        page_no += 1

        if total_count is not None and len(all_rows) >= total_count:
            break

    if not all_rows:
        return pd.DataFrame()

    df = pd.DataFrame(all_rows)

    # Tarih kolonlarını parse et
    for col in df.columns:
        cl = col.lower()
        if cl.startswith("tar") or cl.startswith("date"):
            df[col] = pd.to_datetime(df[col], dayfirst=True, errors="coerce")

    # YEARWEEK varsa ve Tarih yoksa YEARWEEK -> Pazartesi tarihi
    if "YEARWEEK" in df.columns and "Tarih" not in df.columns:
        def yearweek_to_date(val):
            if pd.isna(val):
                return pd.NaT
            s = str(int(val))
            if len(s) < 6:
                s = s.zfill(6)
            year = int(s[:4])
            week = int(s[4:])
            try:
                # ISO week: Pazartesi
                return pd.to_datetime(f"{year}-W{week}-1", format="%G-W%V-%u")
            except Exception:
                return pd.NaT

        df["Tarih"] = df["YEARWEEK"].apply(yearweek_to_date)

    if "Tarih" in df.columns:
        df = df.sort_values("Tarih").reset_index(drop=True)

    return df

def fetch_single_series(
    series_code: str,
    col_name: str,
    start: str,
    end: str,
    frequency: str,
    date_format_value: str,
    verbose: bool = False,
    session: Optional[requests.Session] = None,
    raise_if_empty: bool = True,
) -> pd.DataFrame:
    
    df = evds_serie_market_all(
        series_code=series_code,
        start=start,
        end=end,
        frequency=frequency,
        date_format_value=date_format_value,
        verbose=verbose,
        session=session,
    )

    if df.empty:
        if raise_if_empty:
            raise ValueError(f"{series_code} için veri gelmedi.")
        else:
            if verbose:
                print(f"UYARI: {series_code} için belirtilen aralıkta veri yok (boş DF).")
            out = pd.DataFrame(columns=[col_name])
            out.index = pd.DatetimeIndex([], name="Tarih")
            return out

    if "Tarih" not in df.columns:
        raise ValueError(f"{series_code} için 'Tarih' kolonu yok. Gelen kolonlar: {df.columns.tolist()}")

    non_value_cols = {"Tarih", "YEARWEEK", "UNIXTIME", "totalCount"}
    value_cols = [c for c in df.columns if c not in non_value_cols]

    if len(value_cols) != 1:
        raise ValueError(
            f"{series_code} için beklenenden farklı kolon yapısı: {df.columns.tolist()}"
        )

    val_col = value_cols[0]

    out = (
        df[["Tarih", val_col]]
        .rename(columns={val_col: col_name})
        .set_index("Tarih")
        .sort_index()
    )
    return out
def fetch_weekly_official_from_evds(
    start: str = "01-01-2016",
    end: str | None = None,
    verbose: bool = False,
) -> pd.DataFrame:
    """
    Resmi haftalık veri:
      - TP.AB.A13: .1ba_Kamu ve Diğer Döviz Mevduatı(Bin TL)-Düzey Merkez Bankası Analitik Bilanço (Bin TL)
      - TP.DK.USD.A.YTL: USD/TL alış kuru (gunluk)
      
    """

    if end is None:
        end = datetime.today().strftime("%d-%m-%Y")

    session = requests.Session()
    frequency = "WORKDAY"
    date_format_value = "yyyy-MM-dd"

    common_kwargs = dict(
        start=start,
        end=end,
        frequency=frequency,
        date_format_value=date_format_value,
        verbose=verbose,
        session=session,
        raise_if_empty=True,
    )

   
    usdtry = fetch_single_series(
        series_code="TP.DK.USD.A.YTL",
        col_name="usdtry_daily",
        **common_kwargs,
    )

    # 2) Altın varlık/yükümlülük serileri
    kamu_fx= fetch_single_series(
        series_code="TP.AB.A13",
        col_name="Kamu Mevduatı",
        **common_kwargs,
    )

    # 3) Hepsini tek DF'de birleştir
    kamu_df = (
        usdtry
        .join(kamu_fx, how="inner")
        
        .sort_index()
    )

    
   
    return kamu_df

from datetime import datetime, timedelta



df = fetch_weekly_official_from_evds(start = (datetime.today() - timedelta(days=30)).strftime("%d-%m-%Y")).dropna()



df.to_excel("results/kamu_fx.xlsx")

